<a href="https://colab.research.google.com/github/enigmacy/Image_Captioning/blob/main/Image_captioning03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount= True)
  COLAB = True
  print("Note: using Google CoLab")
  %tensorflow_version 2.x
except:
  COLAB = False
  print("Note: NOT using Google CoLab")

Mounted at /content/drive
Note: using Google CoLab


### **Initializing TPU**

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.7.0
Running on TPU  ['10.68.239.194:8470']
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.68.239.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.68.239.194:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
# Import the Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random
import collections
import json
import string
import os
import glob

from tensorflow.keras.applications.inception_v3 import InceptionV3
import tensorflow.keras.applications.inception_v3

import tensorflow.keras.preprocessing.image
import pickle
from time import time

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (LSTM, Embedding, 
      Dense, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization,
      TimeDistributed, RepeatVector)
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras import Input, layers
from tensorflow.keras import optimizers

from tensorflow.keras.models import Model

from tensorflow.keras.layers import add
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

from PIL import Image
from tqdm import tqdm

In [ ]:
# Runtime
print('GPU', 'Available' if tf.config.list_physical_devices('GPU') else 'Not Available')

NameError: ignored

In [ ]:
# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return f"{h}:{m:>02}:{s:>05.2f}"

In [ ]:
# Direction to the data directory
if COLAB:
    root_captioning = "/content/drive/MyDrive/Image Captioning"
else:
    root_captioning = "./data/captions"

In [ ]:
null_punct = str.maketrans('', '', string.punctuation)
lookup = dict()

with open( os.path.join(root_captioning,'Flicker8K Text',\
                        'Flickr8k.token.txt'), 'r') as fp:
  
  max_length = 0
  for line in fp.read().split('\n'):
    tok = line.split()
    if len(line) >= 2:
      id = tok[0].split('.')[0]
      desc = tok[1:]
      
      # Cleanup description
      desc = [word.lower() for word in desc]
      desc = [w.translate(null_punct) for w in desc]
      desc = [word for word in desc if len(word)>1]
      desc = [word for word in desc if word.isalpha()]
      max_length = max(max_length,len(desc))
      
      if id not in lookup:
        lookup[id] = list()
      lookup[id].append(' '.join(desc))
      
lex = set()
for key in lookup:
  [lex.update(d.split()) for d in lookup[key]]

In [ ]:
print(len(lookup)) # How many unique words
print(len(lex)) # The dictionary
print(max_length) # Maximum length of a caption (in words)

8092
8763
32


In [ ]:
# Saving images direction and file name to a list
img = glob.glob(os.path.join(root_captioning,'Flicker8K Dataset/' + 'Flicker8k_Dataset', '*.jpg'))
len(img)

8091

In [ ]:
# Split into Train and Test sets from the available train and test images text
train_images_path = os.path.join(root_captioning, 'Flicker8K Text', 'Flickr_8k.trainImages.txt')
train_images = set(open(train_images_path, 'r').read().strip().split('\n'))
test_images_path = os.path.join(root_captioning, 'Flicker8K Text', 'Flickr_8k.testImages.txt')
test_images = set(open(test_images_path, 'r').read().strip().split('\n'))

train_img = []
test_img = []

for i in img:
  f = os.path.split(i)[-1]
  if f in train_images:
    train_img.append(f)
  elif f in test_images:
    test_img.append(f)

In [ ]:
print(len(train_img))
print(len(test_img))

6000
1000


In [ ]:
START = 'startseq'
STOP = 'endseq'

train_descriptions = {k: v for k, v in lookup.items() if f'{k}.jpg' in train_images}

for n, v in train_descriptions.items():
  for d in range(len(v)):
    v[d] = f'{START} {v[d]} {STOP}'
  
len(train_descriptions)

6000

In [ ]:
encode_model = InceptionV3(weights= 'imagenet')
encode_model = Model(encode_model.input, encode_model.layers[-2].output)
WIDTH = 299
HEIGHT = 299
OUTPUT_DIM = 2048
preprocess_input = tensorflow.keras.applications.inception_v3.preprocess_input

96124928/96112376 [==============================] - 3s 0us/step


In [ ]:
encode_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                             

In [ ]:
def encodeImage(img):
  img = img.resize((WIDTH, HEIGHT), Image.ANTIALIAS)

  x = tensorflow.keras.preprocessing.image.img_to_array(img)

  x = np.expand_dims(x, axis= 0)

  x = preprocess_input(x)

  x = encode_model.predict(x)

  x = np.reshape(x, OUTPUT_DIM)

  return x

In [ ]:
train_path = os.path.join(root_captioning, 'Data', f'train{OUTPUT_DIM}.pkl')

if not os.path.exists(train_path):
  start = time()
  encoding_train = {}
  for id in tqdm(train_img):
    image_path = os.path.join(root_captioning, 'Flicker8K Dataset/' + 'Flicker8k_Dataset', id)
    img = tensorflow.keras.preprocessing.image.load_img(image_path, target_size= (HEIGHT, WIDTH))
    encoding_train[id] = encodeImage(img)
  with open(train_path, 'wb') as fp:
    pickle.dump(encoding_train, fp)
  print(f"\nGenerating training set took: {hms_string(time()-start)}")
else:
  with open(train_path, 'rb') as fp:
    encoding_train = pickle.load(fp)

In [ ]:
test_path = os.path.join(root_captioning, "Data", f'test{OUTPUT_DIM}.pkl')
if not os.path.exists(test_path):
  start = time()
  encoding_test = {}
  for id in tqdm(test_img):
    image_path = os.path.join(root_captioning,'Flicker8K Dataset/' + 'Flicker8k_Dataset', id)
    img = tensorflow.keras.preprocessing.image.load_img(image_path, \
                target_size=(HEIGHT, WIDTH))
    encoding_test[id] = encodeImage(img)
  with open(test_path, "wb") as fp:
    pickle.dump(encoding_test, fp)
  print(f"\nGenerating testing set took: {hms_string(time()-start)}")
else:
  with open(test_path, "rb") as fp:
    encoding_test = pickle.load(fp)

In [ ]:
all_train_captions = []
for key, val in train_descriptions.items():
  for cap in val:
    all_train_captions.append(cap)
len(all_train_captions)

30000

In [ ]:
words_count_threshold = 7
words_count = {}
nsent = 0
for sent in all_train_captions:
  nsent += 1
  for words in sent.split(' '):
    words_count[words] = words_count.get(words, 0) + 1

vocab = [w for w in words_count if words_count[w] >= words_count_threshold]
print('preprocessed words %d ==> %d' % (len(words_count), len(vocab)))

preprocessed words 7578 ==> 2056


In [ ]:
idxtoword = {}
wordtoidx = {}

ix = 1
for w in vocab:
  wordtoidx[w] = ix
  idxtoword[ix] = w
  ix += 1

vocab_size = len(idxtoword) + 1
vocab_size

2057

In [ ]:
max_length += 2
print(max_length)

34


In [ ]:
def data_generator(descriptions, photos, wordtoidx, max_length, num_photos_per_batch):
  x1, x2, y = [], [], []
  n = 0
  while True:
    for key, desc_list in descriptions.items():
      n += 1
      photo = photos[key + '.jpg']

      for desc in  desc_list:

        seq = [wordtoidx[word] for word in desc.split(' ') if word in wordtoidx]

        for i in range(1, len(seq)):
          in_seq, out_seq = seq[: i], seq[i]
          in_seq = pad_sequences([in_seq], maxlen= max_length)[0]
          out_seq = to_categorical([out_seq], num_classes= vocab_size)[0]
          x1.append(photo)
          x2.append(in_seq)
          y.append(out_seq)
        if n == num_photos_per_batch:
          yield ([np.array(x1), np.array(x2)], np.array(y))
          x1, x2, y = [], [], []
          n = 0

In [ ]:
glove_dir = os.path.join(root_captioning, 'Glove.6B')
embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding= 'utf-8')

for line in tqdm(f):
  values = line.split()
  words = values[0]
  coefs = np.asarray(values[1: ], dtype= 'float32')
  embeddings_index[words] = coefs

f.close()
print(f'Found {len(embeddings_index)} word vectors.')

400000it [00:15, 25040.09it/s]

Found 400000 word vectors.


In [ ]:
embedding_dim = 100

# Get 100-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in wordtoidx.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

In [ ]:
embedding_matrix.shape

(2057, 100)

In [ ]:
OUTPUT_DIM

2048

In [ ]:
inputs1 = Input(shape= (OUTPUT_DIM,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(512, activation= 'relu')(fe1)
fe3 = Dense(1024, activation= 'relu')(fe2)
fe4 = Dense(512, activation= 'relu')(fe3)
inputs2 = Input(shape= (max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero= True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(512, return_sequences= True)(se2)
se4 = Dropout(0.8)(se3)
se5 = LSTM(512)(se4)
decoder1 = add([fe4, se5])
decoder2 = Dense(512, activation= 'relu')(decoder1)
outputs = Dense(vocab_size, activation= 'softmax')(decoder2)
caption_model = Model(inputs= [inputs1, inputs2], outputs= outputs)

In [ ]:
caption_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 34)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 34, 100)      205700      ['input_3[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['input_2[0][0]']                
                                                                                            

In [ ]:
caption_model.layers[2].set_weights([embedding_matrix])
caption_model.layers[2].trainable = False
caption_model.compile(loss= 'categorical_crossentropy', optimizer= 'adam')

In [ ]:
number_pics_per_batch = 3
steps = len(train_descriptions) // number_pics_per_batch

In [ ]:
checkpoint_path = root_captioning + '/Checkpoints/training_3/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only= True,
                                                 verbose= 1)

In [ ]:
EPOCHS = 12
model_path = os.path.join(root_captioning, 'Data', f'caption-model03.hdf5')
if not os.path.exists(model_path):
  start = time()
  for i in tqdm(range(2 * EPOCHS)):
    generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_batch)
    caption_model.fit(x= generator, epochs= 1, steps_per_epoch= steps, verbose= 1, callbacks= [cp_callback])

  caption_model.optimizer.lr = 4 * 1e-4
  number_pics_per_batch = 6
  steps = len(train_descriptions) // number_pics_per_batch

  for i in range(EPOCHS):
    generator = data_generator(train_descriptions, encoding_train, wordtoidx, max_length, number_pics_per_batch)
    caption_model.fit(x= generator, epochs= 1, steps_per_epoch= steps, verbose= 1, callbacks= [cp_callback])

  caption_model.save_weights(model_path)
  print(f"\Training took: {hms_string(time()-start)}")
else:
  caption_model.load_weights(model_path)

  0%|          | 0/24 [00:00<?, ?it/s]

2000/2000 [==============================] - ETA: 0s - loss: 4.4085
Epoch 00001: saving model to /content/drive/MyDrive/Image Captioning/Checkpoints/training_3/cp.ckpt
2000/2000 [==============================] - 556s 273ms/step - loss: 4.4085


  4%|▍         | 1/24 [09:30<3:38:31, 570.06s/it]

2000/2000 [==============================] - ETA: 0s - loss: 3.7330
Epoch 00001: saving model to /content/drive/MyDrive/Image Captioning/Checkpoints/training_3/cp.ckpt
2000/2000 [==============================] - 548s 274ms/step - loss: 3.7330


  8%|▊         | 2/24 [18:52<3:27:16, 565.30s/it]

...

2000/2000 [==============================] - ETA: 0s - loss: 2.8708
Epoch 00001: saving model to /content/drive/MyDrive/Image Captioning/Checkpoints/training_3/cp.ckpt
2000/2000 [==============================] - 536s 268ms/step - loss: 2.8708


 42%|████▏     | 10/24 [1:31:20<2:06:10, 540.78s/it]

2000/2000 [==============================] - ETA: 0s - loss: 2.8237
Epoch 00001: saving model to /content/drive/MyDrive/Image Captioning/Checkpoints/training_3/cp.ckpt
2000/2000 [==============================] - 535s 267ms/step - loss: 2.8237


 46%|████▌     | 11/24 [1:40:15<1:56:46, 538.99s/it]

 317/2000 [===>..........................] - ETA: 7:29 - loss: 2.8149

In [ ]:
def generateCaption(photo):
  in_text = START
  for i in range(max_length):
    sequence = [wordtoidx[w] for w in in_text.split() if w in wordtoidx]
    sequence = pad_sequences([sequence], maxlen= max_length)
    yhat = caption_model.predict([photo, sequence], verbose= 0)
    yhat = np.argmax(yhat)
    word = idxtoword[yhat]
    in_text += ' ' + word
    if word == STOP:
      break
  final = in_text.split()
  final = final[1: -1]
  final = ' '.join(final)
  return final

In [ ]:
for z in range(2): # set higher to see more examples
  pic = random.choice(list(encoding_test.keys()))
  image = encoding_test[pic].reshape((1,OUTPUT_DIM))
  print(os.path.join(root_captioning,'Flicker8K Dataset/' + 'Flicker8k_Dataset', pic))
  x = plt.imread(os.path.join(root_captioning,'Flicker8K Dataset/' + 'Flicker8k_Dataset', pic))
  plt.imshow(x)
  plt.show()
  print("Caption:",generateCaption(image))
  print("_____________________________________")

In [ ]:
images = glob.glob(os.path.join(root_captioning, 'My Photos', '*.jpg'))
for img in images:
  img = Image.open(img)
  img.load()

  plt.imshow(img, cmap= None)
  plt.show()
  
  img = encodeImage(img).reshape((1, OUTPUT_DIM))
  print(img.shape)
  print("Caption:",generateCaption(img))
  print("_____________________________________")